<h2>モデルの登録とエンドをデプロイします </h2>

In [ ]:
import azureml.core
from azureml.core.model import Model, InferenceConfig
from azureml.core.runconfig import CondaDependencies
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core import Workspace
workspace = Workspace.from_config()

In [ ]:
import torch

<h4>モデルを登録します</h4>

In [ ]:
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "./models",
                       model_name = "text_classifier_lstm",
                       model_framework=Model.Framework.PYTORCH,
                       model_framework_version=torch.__version__,
                       description = "text claasifier by lstm",
                       tags={'name': "text classifier lstm", 'ver': "initial"},
                       workspace = workspace)

<h4>score.py ファイルを準備します</h4>

In [ ]:
%%writefile score.py
from azureml.core.model import Model
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics.functional import accuracy
import pytorch_lightning as pl


def init():
    
    class LitTrainClassifier(pl.LightningModule):

        def training_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self.forward(x)
            loss = F.cross_entropy(y_hat, y)
            self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
            return loss

    class LitValidationClassifier(pl.LightningModule):

        def validation_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self.forward(x)
            y_pred = torch.argmax(y_hat, dim=1)
            loss = F.cross_entropy(y_hat, y)
            acc = accuracy(y_pred, y)
            self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
            self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=False, logger=True)
            return loss

    class LitTestClassifier(pl.LightningModule):

        def test_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self.forward(x)
            y_pred = torch.argmax(y_hat, dim=1)
            loss = F.cross_entropy(y_hat, y)
            acc = accuracy(y_pred, y)
            self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=False, logger=True)
            self.log('test_acc', acc, on_step=False, on_epoch=True, prog_bar=False, logger=True)
            return loss

    class LSTMModel(LitTrainClassifier, LitValidationClassifier, LitTestClassifier):

        def __init__(self, vocab_size=7295, embedding_dim=200, hidden_dim=100, layer_dim=2, output_dim=9, drop_out=0.3):
            super(LSTMModel, self).__init__()

            self.hidden_dim = hidden_dim
            self.layer_dim = layer_dim

            self.lstm = torch.nn.LSTM(input_size = embedding_dim,
                                      hidden_size = hidden_dim,
                                      num_layers = layer_dim,
                                      dropout = drop_out,
                                      batch_first=True)    

            self.fc = nn.Linear(hidden_dim, output_dim)
            self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        def forward(self, x):
            x = self.embeddings(x)        
            lstm_out, _ = self.lstm(x)
            out = self.fc(lstm_out[:, -1, :])
            return out

        def configure_optimizers(self):
            return torch.optim.Adam(self.parameters(), lr=0.01)

        
    global net
    model_filename = '/text_classifier_lstm.pt'
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'models')

    net = LSTMModel()
    net.load_state_dict(torch.load(model_path + model_filename))
    
    
def run(raw_data):
    x = np.array(json.loads(raw_data)['data'], dtype=np.int64)
    x = torch.tensor(x, dtype=torch.int64)
    x = torch.unsqueeze(x, 0)
    
    y_hat = net.forward(x)
    y_pred = torch.argmax(y_hat)
    
    return y_pred.item()

<h4>Conda 環境を指定し、yml ファイルを書出します</h4>

In [ ]:
cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_pip_package("azureml-defaults")
cd.add_pip_package('torch==1.8.1')
cd.add_pip_package('pytorch-lightning==1.3.1')
cd.save_to_file(base_directory='./', conda_file_path='text_classifier_lstm.yml')

print(cd.serialize_to_string())

<h4>Web Service endpoint をデプロイします</h4>
<br>10~15分くらいかかります</br>

In [ ]:
myenv = Environment.from_conda_specification(name="text_classifier_lstm", file_path="./text_classifier_lstm.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=3, 
                                               tags={'name':'text_classifier_lstm', 'framework': 'PyTorch'},
                                               description='text classifier lstm')

service = Model.deploy(workspace=workspace,
                           name='text-classifier-lstm', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig, overwrite=True)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.get_logs())

<h4>Endpoint URI を確認します</h4>

- URI は次のステップで使うのでコピーしておきます

In [ ]:
print(service.scoring_uri)

In [ ]:
scoring_uri = ''